<a href="https://colab.research.google.com/github/ns7900f/linux/blob/master/Partiel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install arabicstopwords nltk tashaphyne camel-tools scikit-learn
!pip install camel-tools
!pip install nltk
!pip install tashaphyne
!pip install camel-tools tashaphyne scikit-learn


import re
import numpy as np
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.tokenizers.word import simple_word_tokenize
from tashaphyne.stemming import ArabicLightStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import files
from IPython.display import display
import ipywidgets as widgets
import pandas as pd

# رفع ملف التوقفات (Stop Words)
print("Please upload the Arabic Stop Words documents file (CSV format).")
uploaded = files.upload()

# قراءة ملف التوقفات
stop_words = []
for file_name in uploaded.keys():
    print(f'File uploaded: {file_name}')
    with open(file_name, 'r', encoding='utf-8', errors="replace") as file:
        stop_words = [line.strip() for line in file]

# رفع ملف المستندات العربية (CSV)
print("Please upload the Arabic documents file (CSV format).")
uploaded_documents = files.upload()

# قائمة الترميزات المحتملة
encodings = ["utf-8", "windows-1256", "ISO-8859-6"]

# قراءة ملف CSV باستخدام ترميز صالح
documents = []
for file_name in uploaded_documents.keys():
    if file_name.endswith(".csv"):
        for encoding in encodings:
            try:
                with open(file_name, "r", encoding=encoding) as f:
                    df = pd.read_csv(f, sep=",")

                print(f"✅ Successfully read the file with encoding: {encoding}")

                # التحقق من وجود عمود "Text"
                if 'Text' in df.columns:
                    documents = df['Text'].dropna().tolist()

                    # طباعة النصوص العربية بشكل صحيح
                    for text in documents:
                        print(text)
                else:
                    print("❌ Error: The CSV file must contain a column named 'Text'.")
                    print("Available columns:", df.columns)

                break  # إذا نجح، لا داعي لتجربة المزيد من الترميزات
            except Exception as e:
                print(f"❌ Failed with encoding {encoding}, trying next...")
# Initialize the Arabic light stemmer
stemmer = ArabicLightStemmer()

def preprocess_arabic(text):
    """Preprocess Arabic text by normalizing, removing stop words, and applying stemming."""
    text = normalize_unicode(text)  # Normalize Unicode forms
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)  # Remove non-Arabic characters
    words = simple_word_tokenize(text)  # Tokenize text
    filtered_words = [word for word in words if word not in stop_words]  # Remove stopwords
    stemmed_words = [stemmer.light_stem(word) for word in filtered_words]  # Apply stemming
    return " ".join(stemmed_words)

def compute_tfidf(documents):
    """Compute TF-IDF for Arabic documents."""
    preprocessed_docs = [preprocess_arabic(doc) for doc in documents]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(preprocessed_docs)
    feature_names = vectorizer.get_feature_names_out()
    return tfidf_matrix, feature_names


tfidf_matrix, feature_names = compute_tfidf(documents)

# Convert TF-IDF matrix to a Pandas DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
#tfidf_df = pd.DataFrame(np.round(tfidf_matrix.toarray(), 3), columns=feature_names)
#3 numbers after ,


# Print TF-IDF matrix
print("\nTF-IDF Matrix:")
print(tfidf_df)

# Optionally, save the TF-IDF matrix to a CSV file
tfidf_df.to_csv("tfidf_results.csv", index=False, encoding='utf-8-sig')





# Créer un champ de texte avec une largeur réduite
text_input = widgets.Text(
    placeholder="Please enter a valid Arabic sentence.",
    layout=widgets.Layout(width="300px")  # Ajuste la largeur ici
)

display(text_input)

def on_submit(change):
    sentence = text_input.value.strip()
    if sentence.lower() == 'exit':
        print("Exiting stemming test.")
        text_input.disabled = True  # Désactiver l'entrée utilisateur
        return
    if re.match(r'^[\u0600-\u06FF\s]+$', sentence):
        words = sentence.split()
        #if sentence.lower() == 'exit':
        #print("Exiting stemming test.")
        #return
    #if re.match(r'^[\u0600-\u06FF\s]+$', sentence):
        #words = sentence.split()
        stemmed_words = [stemmer.light_stem(word) for word in words]
        print("\nOriginal Sentence:", sentence)
        print("Stemmed Sentence:", " ".join(stemmed_words))
    else:
        print("❌ Error: Please enter a valid Arabic sentence.")

# Exécuter la fonction lorsqu'on appuie sur Entrée
text_input.on_submit(on_submit)

ERROR: Could not find a version that satisfies the requirement arabicstopwords (from versions: none)
ERROR: No matching distribution found for arabicstopwords
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.5/124.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 857.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [ ]:
# Install required packages
!pip install pandas arabicstopwords nltk camel-tools scikit-learn

import re
import numpy as np
import pandas as pd
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.tokenizers.word import simple_word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import files
from IPython.display import display
import ipywidgets as widgets
from nltk.stem.isri import ISRIStemmer  # Import ISRI Stemmer

# رفع ملف التوقفات (Stop Words)
print("Please upload the Arabic Stop Words file (TXT format).")
uploaded = files.upload()

# قراءة ملف التوقفات
stop_words = []
for file_name in uploaded.keys():
    print(f'File uploaded: {file_name}')
    with open(file_name, 'r', encoding='utf-8', errors="replace") as file:
        stop_words = [line.strip() for line in file]

# رفع ملف المستندات العربية (CSV)
print("Please upload the Arabic documents file (CSV format).")
uploaded_documents = files.upload()

# قائمة الترميزات المحتملة
encodings = ["utf-8", "windows-1256", "ISO-8859-6"]

# قراءة ملف CSV باستخدام ترميز صالح
documents = []
for file_name in uploaded_documents.keys():
    if file_name.endswith(".csv"):
        for encoding in encodings:
            try:
                df = pd.read_csv(file_name, encoding=encoding)
                print(f"✅ Successfully read the file with encoding: {encoding}")

                if 'Text' in df.columns:
                    documents = df['Text'].dropna().tolist()
                    for text in documents:
                        print(text)
                else:
                    print("❌ Error: The CSV file must contain a column named 'Text'.")
                    print("Available columns:", df.columns)
                break
            except Exception:
                print(f"❌ Failed with encoding {encoding}, trying next...")

# Initialize ISRI Stemmer
stemmer = ISRIStemmer()

def preprocess_arabic(text):
    """Preprocess Arabic text by normalizing, removing stop words, and applying ISRI stemming."""
    text = normalize_unicode(text)
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)  # Remove non-Arabic characters
    words = simple_word_tokenize(text)
    filtered_words = [word for word in words if word not in stop_words]
    stemmed_words = [stemmer.stem(word) for word in filtered_words]  # Apply ISRI stemming
    return " ".join(stemmed_words)

def compute_tfidf(documents):
    """Compute TF-IDF for Arabic documents."""
    preprocessed_docs = [preprocess_arabic(doc) for doc in documents]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(preprocessed_docs)
    feature_names = vectorizer.get_feature_names_out()
    return tfidf_matrix, feature_names

tfidf_matrix, feature_names = compute_tfidf(documents)

# Convert TF-IDF matrix to a Pandas DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Print TF-IDF matrix
print("\nTF-IDF Matrix:")
print(tfidf_df)

# Optionally, save the TF-IDF matrix to a CSV file
tfidf_df.to_csv("tfidf_results.csv", index=False, encoding='utf-8-sig')



# User input with widget
text_input = widgets.Text(
    placeholder="Please enter a valid Arabic sentence.",
    layout=widgets.Layout(width="300px")
)
display(text_input)

def on_submit(change):
    sentence = text_input.value.strip()
    if sentence.lower() == 'exit':
        print("Exiting stemming test.")
        text_input.disabled = True  # Disable user input
        return
    if re.match(r'^[\u0600-\u06FF\s]+$', sentence):
        words = sentence.split()
        stemmed_words = [stemmer.stem(word) for word in words]  # Use ISRI Stemmer
        print("\nOriginal Sentence:", sentence)
        print("Stemmed Sentence:", " ".join(stemmed_words))
    else:
        print("❌ Error: Please enter a valid Arabic sentence.")

# Run function when Enter is pressed
text_input.on_submit(on_submit)


ERROR: Could not find a version that satisfies the requirement arabicstopwords (from versions: none)
ERROR: No matching distribution found for arabicstopwords
Please upload the Arabic Stop Words file (TXT format).


Saving stop_words.csv to stop_words (4).csv
File uploaded: stop_words (4).csv
Please upload the Arabic documents file (CSV format).


Saving documents.csv to documents (4).csv
❌ Failed with encoding utf-8, trying next...
✅ Successfully read the file with encoding: windows-1256
هذا هو النص الأول يحتوي على كلمات عربية
النص الثاني يحتوي على العديد من الكلمات المختلفة
النص الثالث بسيط ولا يحتوي على كلمات كثيرة
لعب سامي مع احمد في الطابة
 تلبس صديقتي على راسها قبعة   مع شال ملتف حول عنقها 

TF-IDF Matrix:
        اول       بسط       ثلث       ثني  حمد       حول       حوي       خلف  \
0  0.546722  0.000000  0.000000  0.000000  0.0  0.000000  0.366146  0.000000   
1  0.000000  0.000000  0.000000  0.479709  0.0  0.000000  0.321267  0.479709   
2  0.000000  0.432518  0.432518  0.000000  0.0  0.000000  0.289662  0.000000   
3  0.000000  0.000000  0.000000  0.000000  0.5  0.000000  0.000000  0.000000   
4  0.000000  0.000000  0.000000  0.000000  0.0  0.353553  0.000000  0.000000   

        راس  سمي  ...       عرب       عنق       قبع       كثر       كلم  \
0  0.000000  0.0  ...  0.546722  0.000000  0.000000  0.000000  0.366146 

Text(value='', layout=Layout(width='300px'), placeholder='Please enter a valid Arabic sentence.')

In [ ]:
!pip install pandas
# Install required packages
!pip install arabicstopwords nltk tashaphyne camel-tools scikit-learn
!pip install camel-tools
!pip install nltk
!pip install tashaphyne
!pip install camel-tools tashaphyne scikit-learn


import re
import numpy as np
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.tokenizers.word import simple_word_tokenize
from tashaphyne.stemming import ArabicLightStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import files
from IPython.display import display
import ipywidgets as widgets
import pandas as pd

# رفع ملف التوقفات (Stop Words)
print("Please upload the Arabic Stop Words documents file (CSV format).")
uploaded = files.upload()

# قراءة ملف التوقفات
stop_words = []
for file_name in uploaded.keys():
    print(f'File uploaded: {file_name}')
    with open(file_name, 'r', encoding='utf-8', errors="replace") as file:
        stop_words = [line.strip() for line in file]

# رفع ملف المستندات العربية (CSV)
print("Please upload the Arabic documents file (CSV format).")
uploaded_documents = files.upload()

# قائمة الترميزات المحتملة
encodings = ["utf-8", "windows-1256", "ISO-8859-6"]

# قراءة ملف CSV باستخدام ترميز صالح
documents = []
for file_name in uploaded_documents.keys():
    if file_name.endswith(".csv"):
        for encoding in encodings:
            try:
                with open(file_name, "r", encoding=encoding) as f:
                    df = pd.read_csv(f, sep=",")

                print(f"✅ Successfully read the file with encoding: {encoding}")

                # التحقق من وجود عمود "Text"
                if 'Text' in df.columns:
                    documents = df['Text'].dropna().tolist()

                    # طباعة النصوص العربية بشكل صحيح
                    for text in documents:
                        print(text)
                else:
                    print("❌ Error: The CSV file must contain a column named 'Text'.")
                    print("Available columns:", df.columns)

                break  # إذا نجح، لا داعي لتجربة المزيد من الترميزات
            except Exception as e:
                print(f"❌ Failed with encoding {encoding}, trying next...")
# Initialize the Arabic light stemmer
stemmer = ArabicLightStemmer()

def preprocess_arabic(text):
    """Preprocess Arabic text by normalizing, removing stop words, and applying stemming."""
    text = normalize_unicode(text)  # Normalize Unicode forms
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)  # Remove non-Arabic characters
    words = simple_word_tokenize(text)  # Tokenize text
    filtered_words = [word for word in words if word not in stop_words]  # Remove stopwords
    stemmed_words = [stemmer.light_stem(word) for word in filtered_words]  # Apply stemming
    return " ".join(stemmed_words)

def compute_tfidf(documents):
    """Compute TF-IDF for Arabic documents."""
    preprocessed_docs = [preprocess_arabic(doc) for doc in documents]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(preprocessed_docs)
    feature_names = vectorizer.get_feature_names_out()
    return tfidf_matrix, feature_names


# Compute TF-IDF
tfidf_matrix, feature_names = compute_tfidf(documents)

# Print TF-IDF values with corresponding words
print("\nTF-IDF Scores (Word -> Score per Document):")
for i, doc_vector in enumerate(tfidf_matrix.toarray()):
    print(f"\nDocument {i+1}:")
    for word, score in zip(feature_names, doc_vector):
        if score > 0:  # Print only words with non-zero scores
            print(f"  {word}: {score:.4f}")



# Créer un champ de texte avec une largeur réduite
text_input = widgets.Text(
    placeholder="Please enter a valid Arabic sentence.",
    layout=widgets.Layout(width="300px")  # Ajuste la largeur ici
)

display(text_input)

def on_submit(change):
    sentence = text_input.value.strip()
    if sentence.lower() == 'exit':
        print("Exiting stemming test.")
        text_input.disabled = True  # Désactiver l'entrée utilisateur
        return
    if re.match(r'^[\u0600-\u06FF\s]+$', sentence):
        words = sentence.split()
        #if sentence.lower() == 'exit':
        #print("Exiting stemming test.")
        #return
    #if re.match(r'^[\u0600-\u06FF\s]+$', sentence):
        #words = sentence.split()
        stemmed_words = [stemmer.light_stem(word) for word in words]
        print("\nOriginal Sentence:", sentence)
        print("Stemmed Sentence:", " ".join(stemmed_words))
    else:
        print("❌ Error: Please enter a valid Arabic sentence.")

# Exécuter la fonction lorsqu'on appuie sur Entrée
text_input.on_submit(on_submit)


ERROR: Could not find a version that satisfies the requirement arabicstopwords (from versions: none)
ERROR: No matching distribution found for arabicstopwords
Please upload the Arabic Stop Words documents file (CSV format).


Saving stop_words.csv to stop_words (5).csv
File uploaded: stop_words (5).csv
Please upload the Arabic documents file (CSV format).


Saving documents.csv to documents (5).csv
❌ Failed with encoding utf-8, trying next...
✅ Successfully read the file with encoding: windows-1256
هذا هو النص الأول يحتوي على كلمات عربية
النص الثاني يحتوي على العديد من الكلمات المختلفة
النص الثالث بسيط ولا يحتوي على كلمات كثيرة
لعب سامي مع احمد في الطابة
 تلبس صديقتي على راسها قبعة   مع شال ملتف حول عنقها 

TF-IDF Scores (Word -> Score per Document):

Document 1:
  أول: 0.5309
  حتوي: 0.3555
  عرب: 0.5309
  لم: 0.4283
  نص: 0.3555

Document 2:
  ثان: 0.4519
  حتوي: 0.3026
  عديد: 0.4519
  كلم: 0.4519
  مختلف: 0.4519
  نص: 0.3026

Document 3:
  ثالث: 0.4246
  ثير: 0.4246
  حتوي: 0.2843
  سيط: 0.4246
  لا: 0.4246
  لم: 0.3425
  نص: 0.2843

Document 4:
  حمد: 0.5000
  سام: 0.5000
  طاب: 0.5000
  عب: 0.5000

Document 5:
  حول: 0.3536
  راس: 0.3536
  شال: 0.3536
  صديق: 0.3536
  عنق: 0.3536
  قبع: 0.3536
  لبس: 0.3536
  ملتف: 0.3536


Text(value='', layout=Layout(width='300px'), placeholder='Please enter a valid Arabic sentence.')

❌ Error: Please enter a valid Arabic sentence.

Original Sentence: ركضت نحو الباب
Stemmed Sentence: ركض نحو باب

Original Sentence: ركضت
Stemmed Sentence: ركض
Exiting stemming test.
